In [1]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
movies_df = pd.read_table('recommenders/data/movies.dat', header=None, sep='::', 
                          names=['movie_id', 'movie_title', 'movie_genre'], engine='python')
movies_df = pd.concat([movies_df, movies_df.movie_genre.str.get_dummies(sep='|')], axis=1)  

In [6]:
movie_categories = movies_df.columns[3:].tolist()

In [8]:
movies_df.head()

,movie_id,movie_title,movie_genre,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


movies_df.head(10)

In [9]:
movie_categories

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [10]:
movies_df.sample(10)

,movie_id,movie_title,movie_genre,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
314,317,"Santa Clause, The (1994)",Children's|Comedy|Fantasy,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
149,151,Rob Roy (1995),Drama|Romance|War,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
612,616,"Aristocats, The (1970)",Animation|Children's,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3481,3550,"Hunger, The (1983)",Horror,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2388,2457,Running Scared (1986),Action|Comedy,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1137,1153,Raw Deal (1948),Film-Noir,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1176,1193,One Flew Over the Cuckoo's Nest (1975),Drama,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3581,3650,Anguish (Angustia) (1986),Horror,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1201,1219,Psycho (1960),Horror|Thriller,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1969,2038,"Cat from Outer Space, The (1978)",Children's|Comedy|Sci-Fi,0,0,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [15]:
ratings_df = pd.read_table('recommenders/data/ratings.dat', header=None, sep='::', 
                          names=['user_id', 'movie_id', 'rating', 'timestamp'], engine='python')

In [16]:
#we dont care about the time the rating was given
del ratings_df['timestamp']

In [17]:
#replace movie_id with movie_title for legibility
ratings_df = pd.merge(ratings_df, movies_df, on='movie_id')[['user_id', 'movie_title', 'movie_id','rating', 'movie_genre']]

ratings_df.head()

,user_id,movie_title,movie_id,rating,movie_genre
0,1,One Flew Over the Cuckoo's Nest (1975),1193,5,Drama
1,2,One Flew Over the Cuckoo's Nest (1975),1193,5,Drama
2,12,One Flew Over the Cuckoo's Nest (1975),1193,4,Drama
3,15,One Flew Over the Cuckoo's Nest (1975),1193,4,Drama
4,17,One Flew Over the Cuckoo's Nest (1975),1193,5,Drama


In [18]:
ratings_mtx_df = ratings_df.pivot_table(values='rating', index='user_id', columns='movie_title')  

ratings_mtx_df = ratings_mtx_df.apply(lambda x: x.fillna(x.mean()),axis=0)
ratings_mtx_df = ratings_mtx_df.apply(lambda x: x - x.mean(),axis=1)

movie_index = ratings_mtx_df.columns

ratings_mtx_df.head()

movie_title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),...,"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kj�rlighetens kj�tere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
1,-0.215674,0.128728,-0.550393,-0.331810,0.470867,-0.742701,0.180156,0.353760,-0.195997,1.052754,...,0.390211,0.147801,0.057299,0.133446,0.257299,0.171092,0.508130,0.257299,-0.720962,0.013397
2,-0.210064,0.134337,-0.544784,-0.326200,0.476476,-0.737091,0.185766,0.359369,-0.190388,1.058363,...,0.395820,0.153410,0.062909,0.139055,0.262909,0.176702,0.513739,0.262909,-0.715352,0.019006
3,-0.213476,0.130926,-0.548195,-0.329612,0.473065,-0.740503,0.182354,0.355957,-0.193799,1.054952,...,0.392409,0.149999,0.059497,0.135644,0.259497,0.173290,0.510328,0.259497,-0.718764,0.015595
4,-0.212675,0.131726,-0.547394,-0.328811,0.473866,-0.739702,0.183155,0.356758,-0.192999,1.055752,...,0.393209,0.150799,0.060298,0.136445,0.260298,0.174091,0.511128,0.260298,-0.717963,0.016395
5,-0.182296,0.162105,-0.517016,-0.298432,0.504244,-0.709323,0.213534,0.387137,-0.162620,1.086131,...,0.423588,0.181178,0.090677,0.166823,0.290677,0.204470,0.541507,0.290677,-0.687584,0.046774


In [20]:
corr_matrix = np.corrcoef(ratings_mtx_df.T)  # Pearson product­moment correlation coefficient (PPMC)
corr_matrix.shape  # correlation between movies

(3706, 3706)

In [21]:
corr_matrix[0]

array([ 1.        ,  0.03387902,  0.05688379, ...,  0.26638272,
        0.10525703, -0.02484298])

In [22]:
movie_index

Index(['$1,000,000 Duck (1971)', ''Night Mother (1986)',
       ''Til There Was You (1997)', ''burbs, The (1989)',
       '...And Justice for All (1979)', '1-900 (1994)',
       '10 Things I Hate About You (1999)', '101 Dalmatians (1961)',
       '101 Dalmatians (1996)', '12 Angry Men (1957)',
       ...
       'Young Poisoner's Handbook, The (1995)', 'Young Sherlock Holmes (1985)',
       'Young and Innocent (1937)', 'Your Friends and Neighbors (1998)',
       'Zachariah (1971)', 'Zed & Two Noughts, A (1985)', 'Zero Effect (1998)',
       'Zero Kelvin (Kj�rlighetens kj�tere) (1995)', 'Zeus and Roxanne (1997)',
       'eXistenZ (1999)'],
      dtype='object', name='movie_title', length=3706)

In [23]:
def get_movie_correlations(movie_title):  
    '''Returns correlation vector for a movie'''
    movie_idx = list(movie_index).index(movie_title)
    return corr_matrix[movie_idx]

def get_similar_movies(movie_title, threshold=0.2):
    movie_correlations_array =  get_movie_correlations(movie_title)
    return movie_index[movie_correlations_array>threshold]

In [24]:
ratings_mtx_df.T

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movie_title,,,,,,,,,,,,,,,,,,,,,
"$1,000,000 Duck (1971)",-0.215674,-0.210064,-0.213476,-0.212675,-0.182296,-0.217548,-0.216115,-0.221737,-0.208101,-0.262994,...,-0.210600,-0.217917,-0.213876,-0.212583,-0.164316,-0.143992,-0.197548,-0.210879,-0.209104,-0.184900
'Night Mother (1986),0.128728,0.134337,0.130926,0.131726,0.162105,0.126854,0.128286,0.122664,0.136300,0.081407,...,0.133802,0.126485,0.130525,0.131818,0.180086,-0.171019,0.146853,0.133523,0.135298,0.159501
'Til There Was You (1997),-0.550393,-0.544784,-0.548195,-0.547394,-0.517016,-0.552267,-0.550835,-0.556457,-0.542821,-0.597714,...,-0.545319,-0.552636,-0.548596,-0.547303,-0.499035,-0.478711,-0.532268,-0.545598,-0.543823,-0.519620
"'burbs, The (1989)",-0.331810,-0.326200,-0.329612,-0.328811,-0.298432,-0.333684,-0.332251,-0.337873,-0.324237,0.709979,...,-0.326736,-0.334053,-0.330012,-0.328719,-0.280452,-0.260128,-0.313684,-0.327015,-0.325240,-0.301036
...And Justice for All (1979),0.470867,0.476476,0.473065,0.473866,0.504244,0.468993,0.470426,0.464803,0.478439,0.423546,...,0.475941,0.468624,0.472664,0.473958,0.522225,0.542549,0.488992,0.475662,0.477437,0.501641
1-900 (1994),-0.742701,-0.737091,-0.740503,-0.739702,-0.709323,-0.744575,-0.743142,-0.748764,-0.735128,-0.790021,...,-0.737627,-0.744944,-0.740903,-0.739610,-0.691343,-0.671019,-0.724576,-0.737906,-0.736131,-0.711927
10 Things I Hate About You (1999),0.180156,0.185766,0.182354,0.183155,0.213534,0.178282,0.179715,0.174093,0.187729,0.132836,...,0.185230,0.177913,0.181954,0.183247,0.231514,-1.171019,0.198282,0.184951,0.186726,0.210930
101 Dalmatians (1961),0.353760,0.359369,0.355957,0.356758,0.387137,0.351885,0.353318,0.347696,0.361332,0.306439,...,0.358833,0.755056,0.355557,0.356850,0.405117,0.828981,0.371885,0.358554,0.360329,0.384533
101 Dalmatians (1996),-0.195997,-0.190388,-0.193799,-0.192999,-0.162620,-0.197872,-0.196439,-0.202061,-0.188425,-0.243318,...,-0.190924,-0.198241,-0.194200,-0.192907,-2.191343,-0.124316,-0.177872,-0.191203,-0.189428,-0.165224


In [24]:
corr_matrix

array([[  1.00000000e+00,   3.38790195e-02,   5.68837908e-02, ...,
          2.66382718e-01,   1.05257034e-01,  -2.48429774e-02],
       [  3.38790195e-02,   1.00000000e+00,  -1.32194687e-02, ...,
          1.00183847e-01,   3.26130706e-02,  -1.57568962e-02],
       [  5.68837908e-02,  -1.32194687e-02,   1.00000000e+00, ...,
          1.92041086e-01,   7.85162956e-02,   1.65494058e-04],
       ..., 
       [  2.66382718e-01,   1.00183847e-01,   1.92041086e-01, ...,
          1.00000000e+00,   3.15144002e-01,  -3.21763262e-02],
       [  1.05257034e-01,   3.26130706e-02,   7.85162956e-02, ...,
          3.15144002e-01,   1.00000000e+00,  -1.02228164e-02],
       [ -2.48429774e-02,  -1.57568962e-02,   1.65494058e-04, ...,
         -3.21763262e-02,  -1.02228164e-02,   1.00000000e+00]])

In [25]:
corr_matrix.shape

(3706, 3706)

In [33]:
def get_movie_recommendations(user_movies):  
    '''given a set of movies, it returns all the movies sorted by their correlation with the user'''
    movie_similarities = np.zeros(corr_matrix.shape[0])
    for movie_id in user_movies:
        movie_similarities = movie_similarities + get_movie_correlations(movie_id)
    similarities_df = pd.DataFrame({
        'movie_title': movie_index,
        'sum_similarity': movie_similarities
        })
    similarities_df = similarities_df[~(similarities_df.movie_title.isin(user_movies))]
    similarities_df = similarities_df.sort_values(by=['sum_similarity'], ascending=False)
    return similarities_df

In [34]:
sample_user = 21
sample_user_movies = ratings_df[ratings_df.user_id==sample_user].movie_title.tolist() 
sample_user_movies

["Bug's Life, A (1998)",
 'Bambi (1942)',
 'Antz (1998)',
 'Aladdin (1992)',
 'Toy Story (1995)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Who Framed Roger Rabbit? (1988)',
 'South Park: Bigger, Longer and Uncut (1999)',
 'Akira (1988)',
 'Pinocchio (1940)',
 'Mad Max Beyond Thunderdome (1985)',
 'Titan A.E. (2000)',
 "Devil's Advocate, The (1997)",
 'Prince of Egypt, The (1998)',
 'Wild Wild West (1999)',
 'Iron Giant, The (1999)',
 'Brady Bunch Movie, The (1995)',
 'Princess Mononoke, The (Mononoke Hime) (1997)',
 'Little Nemo: Adventures in Slumberland (1992)',
 'Messenger: The Story of Joan of Arc, The (1999)',
 'Stop! Or My Mom Will Shoot (1992)',
 'House Party 2 (1991)']

In [35]:
recommendations = get_movie_recommendations(sample_user_movies)

recommendations.movie_title.head(20)

3055    Snow White and the Seven Dwarfs (1937)
1865                 Lady and the Tramp (1955)
679                          Cinderella (1950)
1002                              Dumbo (1941)
1939                     Lion King, The (1994)
324                Beauty and the Beast (1991)
7                        101 Dalmatians (1961)
2770           Rescuers Down Under, The (1990)
1948                Little Mermaid, The (1989)
3412                        Toy Story 2 (1999)
2808                         Robin Hood (1973)
3026                    Sleeping Beauty (1959)
1739          James and the Giant Peach (1996)
3275                             Tarzan (1999)
97                  Alice in Wonderland (1951)
1781                   Jungle Book, The (1967)
1611       Hunchback of Notre Dame, The (1996)
2771                      Rescuers, The (1977)
2432                   Oliver & Company (1988)
1111                           Fantasia (1940)
Name: movie_title, dtype: object

In [30]:
recommendations.sum_similarity

3055    1.421961
1865    1.400303
679     1.386220
1002    1.364381
1939    1.289517
324     1.282859
7       1.265600
2770    1.264939
1948    1.200695
3412    1.177436
2808    1.130227
3026    1.112902
1739    1.112877
3275    1.035702
97      1.014506
1781    0.957231
1611    0.944245
2771    0.923160
2432    0.907356
1111    0.901593
1230    0.898925
3154    0.892595
2924    0.886201
141     0.878402
2252    0.869153
3102    0.859501
1007    0.856249
3656    0.850339
1568    0.833654
3153    0.828952
          ...   
1931   -1.685336
2365   -1.685336
3125   -1.685336
2887   -1.685336
981    -1.685336
650    -1.685336
3646   -1.685336
1022   -1.685336
3296   -1.685336
3637   -1.685336
2455   -1.685336
3040   -1.685336
2321   -1.685336
2343   -1.685336
2121   -1.685336
2314   -1.685336
3028   -1.685336
2847   -1.685336
3609   -1.685336
445    -1.685336
667    -1.685336
3487   -1.685336
1787   -1.685336
298    -1.685336
439    -1.685336
2675   -1.685336
1806   -1.685336
914    -1.6853